## Chapter 16: Real‑World Project Structure and Best Practices

Throughout this book, you’ve built a progressively more complex e‑commerce application, learning each facet of .NET Aspire along the way. But the examples have been relatively small—a handful of services, a single team, a straightforward deployment. In the real world, applications grow, teams multiply, and requirements evolve. How do you structure a large‑scale Aspire solution to remain maintainable, versionable, and deployable? What are the best practices for CI/CD, testing, and avoiding common pitfalls?

In this final chapter, we’ll step back and look at the bigger picture. You’ll learn:

- How to organize a solution with multiple AppHosts, shared contracts, and reusable components.
- Strategies for versioning your services, APIs, and configuration.
- How to set up a robust CI/CD pipeline that builds, tests, and deploys your Aspire application.
- Common troubleshooting techniques and pitfalls to avoid.

By the end, you’ll be equipped to apply Aspire to real‑world, enterprise‑scale projects.

---

### 16.1 Organizing a Large‑Scale Aspire Solution

As your application grows, the single‑solution model with one AppHost may become unwieldy. Consider a scenario with:

- Multiple teams owning different sets of services.
- Different release cadences for different parts of the system.
- Shared libraries (e.g., contracts, service defaults) that need versioning.
- Multiple environments (dev, test, staging, production) that may require slightly different resource configurations.

#### 16.1.1 Multiple AppHosts

You can have **multiple AppHost projects** in a single repository, each targeting a subset of services. For example:

- `AppHost.Development`: starts all services and dependencies for full‑stack development.
- `AppHost.PaymentTeam`: starts only payment‑related services and their dependencies.
- `AppHost.IntegrationTests`: a trimmed‑down AppHost for fast test execution.

Each AppHost references the same underlying service projects but may include different resources or configurations. This is achieved by having separate `Program.cs` files that add only the required resources.

**Example directory structure:**

```
/src
  /Services
    /PaymentService
    /OrderService
    /InventoryService
  /AppHosts
    /AppHost.Dev
    /AppHost.Payment
    /AppHost.Testing
  /ServiceDefaults
  /Contracts
```

Each AppHost has its own project file that references the necessary service projects and the `ServiceDefaults` project. This keeps the orchestration layer flexible.

#### 16.1.2 Shared Service Defaults

You may have multiple service default projects if different parts of your organization have different requirements. For instance:

- `ServiceDefaults.Core`: basic telemetry, health checks, and resilience.
- `ServiceDefaults.Azure`: adds Azure‑specific components (like Key Vault integration).
- `ServiceDefaults.PCI`: adds additional security and compliance checks.

Services can then reference the appropriate defaults. This modularity prevents a single monolithic defaults project from becoming a dumping ground.

#### 16.1.3 Contracts and Shared Models

When services communicate, they often share data contracts (e.g., DTOs, event schemas). It’s best to put these in a separate **Contracts** project that is versioned independently. This project can be published as a NuGet package, allowing multiple services (even across repositories) to reference the same contract types.

```xml
<!-- Contracts.csproj -->
<Project Sdk="Microsoft.NET.Sdk">
  <PropertyGroup>
    <TargetFramework>net8.0</TargetFramework>
    <Nullable>enable</Nullable>
  </PropertyGroup>
</Project>
```

Then, in each service:

```xml
<ProjectReference Include="../../Contracts/Contracts.csproj" />
```

Or, if using NuGet:

```xml
<PackageReference Include="MyCompany.ECommerce.Contracts" Version="1.2.3" />
```

#### 16.1.4 Component Libraries

If you develop custom Aspire components internally (e.g., for your own databases or legacy systems), place them in a dedicated `Components` folder and package them as NuGet. This promotes reuse across teams and solutions.

---

### 16.2 Versioning Strategies

Versioning is critical when you have multiple services that evolve independently. You need to version:

- **APIs**: the HTTP/gRPC interfaces between services.
- **Messages**: the event schemas.
- **Components**: the NuGet packages your services depend on.
- **The overall application**: for release tracking.

#### 16.2.1 API Versioning

Use ASP.NET Core API versioning to manage changes to your HTTP APIs. There are several approaches: URL path versioning (e.g., `/api/v1/products`), query string versioning, or header versioning. Choose one and be consistent.

Example with URL path versioning:

```csharp
// In Program.cs
builder.Services.AddApiVersioning(options =>
{
    options.DefaultApiVersion = new ApiVersion(1, 0);
    options.AssumeDefaultVersionWhenUnspecified = true;
    options.ReportApiVersions = true;
    options.ApiVersionReader = new UrlSegmentApiVersionReader();
})
.AddApiExplorer(options =>
{
    options.GroupNameFormat = "'v'VVV";
    options.SubstituteApiVersionInUrl = true;
});
```

Then in controllers:

```csharp
[ApiVersion("1.0")]
[Route("api/v{version:apiVersion}/[controller]")]
public class ProductsController : ControllerBase
```

#### 16.2.2 Message Versioning

For events, use a schema registry or include a version number in the message envelope. For example:

```json
{
  "eventType": "OrderCreated",
  "version": "1.0.0",
  "data": { ... }
}
```

Consumers can then handle different versions appropriately. Consider using **CloudEvents** standard.

#### 16.2.3 Component Versioning

All shared NuGet packages (contracts, components, service defaults) should follow [Semantic Versioning](https://semver.org/). This allows services to upgrade independently as long as the major version matches.

In your CI pipeline, you can automatically version packages based on git tags or build numbers.

---

### 16.3 CI/CD Pipelines

A robust CI/CD pipeline is essential for frequent, reliable deployments. Let’s build a pipeline that:

- Builds all services and runs unit tests.
- Runs integration tests using the `Aspire.Hosting.Testing` approach.
- Builds and pushes container images.
- Deploys to a target environment (e.g., Azure Container Apps).

We’ll use **GitHub Actions** as an example, but the concepts apply to Azure DevOps, GitLab CI, etc.

#### 16.3.1 Building and Testing

Create a workflow file `.github/workflows/ci.yml`:

```yaml
name: CI

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  build:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v3
    - name: Setup .NET
      uses: actions/setup-dotnet@v3
      with:
        dotnet-version: 8.0.x
    - name: Restore
      run: dotnet restore
    - name: Build
      run: dotnet build --no-restore
    - name: Test
      run: dotnet test --no-build --verbosity normal
```

This runs unit tests but not integration tests (which require Docker). For integration tests, we need a runner that supports Docker. GitHub Actions hosted runners have Docker installed.

Add a step to run integration tests:

```yaml
    - name: Integration Tests
      run: dotnet test MyAspireApp.Tests --configuration Release --no-build
```

Make sure your integration tests are designed to be run in CI: they should use the same AppHost but may need to pull container images. The first run may be slow, but subsequent runs are cached.

#### 16.3.2 Building and Pushing Container Images

You can build images using `docker build` or use `azd` to handle it. For GitHub Actions, you might want to push to a container registry (e.g., Docker Hub, ACR).

First, log in to the registry:

```yaml
    - name: Log in to Azure Container Registry
      uses: azure/docker-login@v1
      with:
        login-server: myregistry.azurecr.io
        username: ${{ secrets.ACR_USERNAME }}
        password: ${{ secrets.ACR_PASSWORD }}
```

Then build and push images for each service. You can use a script or a matrix strategy.

Alternatively, use `azd` to handle the entire deployment, including building and pushing.

#### 16.3.3 Deploying with AZD

If you’re using Azure Container Apps, `azd` is the simplest deployment tool. In your pipeline, you can install `azd` and run `azd up` or `azd deploy`.

```yaml
    - name: Install azd
      uses: Azure/setup-azd@v1

    - name: Deploy
      env:
        AZURE_CLIENT_ID: ${{ secrets.AZURE_CLIENT_ID }}
        AZURE_TENANT_ID: ${{ secrets.AZURE_TENANT_ID }}
        AZURE_SUBSCRIPTION_ID: ${{ secrets.AZURE_SUBSCRIPTION_ID }}
      run: azd up --environment production
```

You’ll need to set up a service principal for authentication and store its credentials in GitHub secrets.

#### 16.3.4 Deploying with aspirate

If you’re deploying to Kubernetes, you can use `aspirate` to generate manifests and then `kubectl` to apply them. Example:

```yaml
    - name: Install aspirate
      run: dotnet tool install -g aspirate

    - name: Generate Kubernetes manifests
      run: aspirate generate --manifest-path ./aspire-manifest.json --output-path ./manifests

    - name: Deploy to Kubernetes
      run: kubectl apply -f ./manifests/
```

You’ll need to configure `kubectl` with the appropriate kubeconfig.

#### 16.3.5 Running Migrations in Pipelines

Database migrations should be run as part of the deployment. You can include a migration step that runs the migration resource or a separate job. With `azd`, you can add a hook to run migrations after deployment.

---

### 16.4 Pitfalls to Avoid

Even with Aspire’s simplicity, certain issues can trip you up. Here are common pitfalls and how to avoid them.

#### 16.4.1 Service Discovery Failures

- **Problem**: A service can’t reach another because the logical name is misspelled or the environment variables aren’t set.
- **Solution**: Always use the exact logical name defined in the AppHost. Check that `WithReference` is used correctly. In Kubernetes, ensure that the service names match (they should, because `aspirate` names them after the resource).

#### 16.4.2 Container Networking

- **Problem**: Containers can’t communicate because they’re on different networks.
- **Solution**: In the AppHost, all containers are on the same default network, so this shouldn’t happen. In Kubernetes, services are discoverable via DNS. In ACA, internal DNS works within the environment.

#### 16.4.3 Resource Naming Conflicts

- **Problem**: Two resources with the same logical name in different AppHosts cause confusion.
- **Solution**: Use unique prefixes or namespaces. In large organizations, adopt a naming convention (e.g., `team-product-service`).

#### 16.4.4 Parameter Handling in CI

- **Problem**: CI builds fail because parameters (like passwords) are not provided.
- **Solution**: Use environment variables or secrets in the CI system. In `azd`, parameters can be set in `.env` files or passed via `--set` flag. In GitHub Actions, store secrets and map them to environment variables.

#### 16.4.5 Health Checks Not Configured

- **Problem**: Services appear healthy in the dashboard but fail in production because health checks aren’t mapped.
- **Solution**: Always map health check endpoints in your services, and configure liveness/readiness probes in your production orchestrator.

#### 16.4.6 Inefficient Dockerfiles

- **Problem**: Dockerfiles that copy entire solutions and rebuild everything slow down CI.
- **Solution**: Use multi‑stage builds and leverage Docker layer caching. Organize your Dockerfiles to copy only the necessary project files first, restore, then copy the rest.

#### 16.4.7 Not Using IHostedService for Workers

- **Problem**: Worker services that don’t implement `BackgroundService` correctly may exit prematurely.
- **Solution**: Ensure your worker services derive from `BackgroundService` and have an infinite loop or wait for cancellation.

---

### 16.5 Hands‑on: Setting Up a Multi‑AppHost Solution

Let’s restructure our e‑commerce solution to demonstrate these concepts. We’ll create two AppHosts: one for full development (`AppHost.Dev`) and one for the payment team (`AppHost.Payment`).

#### Step 1: Create the AppHost Projects

In the solution root, create a folder `AppHosts`. Inside, create two new ASP.NET Core Empty projects (targeting .NET 8) named `AppHost.Dev` and `AppHost.Payment`. Add the Aspire.Hosting package to each.

```bash
dotnet new web -n AppHost.Dev -f net8.0
dotnet new web -n AppHost.Payment -f net8.0
cd AppHost.Dev
dotnet add package Aspire.Hosting
dotnet add reference ../../MyAspireApp.ServiceDefaults  # if needed
```

Do the same for `AppHost.Payment`.

#### Step 2: Configure AppHost.Dev

In `AppHost.Dev/Program.cs`, add all services and dependencies:

```csharp
using Aspire.Hosting;

var builder = DistributedApplication.CreateBuilder(args);

// Infrastructure
var postgres = builder.AddPostgres("postgres").WithDataVolume();
var productsDb = postgres.AddDatabase("productsdb");
var rabbitMq = builder.AddRabbitMQ("messaging").WithManagementPlugin();
var storage = builder.AddAzureStorage("storage").RunAsEmulator();
var blobs = storage.AddBlobs("productimages");
var keycloak = builder.AddKeycloak("keycloak", 8080).WithDataVolume();

// Services
var apiService = builder.AddProject<Projects.MyAspireApp_ApiService>("apiservice")
    .WithReference(productsDb)
    .WithReference(rabbitMq)
    .WithReference(blobs)
    .WithReference(keycloak);

var workerService = builder.AddProject<Projects.MyAspireApp_WorkerService>("workerservice")
    .WithReference(rabbitMq)
    .WithReference(productsDb);

var webFrontend = builder.AddProject<Projects.MyAspireApp_Web>("webfrontend")
    .WithReference(apiService)
    .WithReference(keycloak);

builder.Build().Run();
```

#### Step 3: Configure AppHost.Payment

This AppHost might only include payment‑related services. For example, assume we have a `PaymentService` and it depends only on RabbitMQ and PostgreSQL.

```csharp
using Aspire.Hosting;

var builder = DistributedApplication.CreateBuilder(args);

var postgres = builder.AddPostgres("postgres").WithDataVolume();
var paymentsDb = postgres.AddDatabase("paymentsdb");
var rabbitMq = builder.AddRabbitMQ("messaging").WithManagementPlugin();

var paymentService = builder.AddProject<Projects.MyAspireApp_PaymentService>("paymentservice")
    .WithReference(paymentsDb)
    .WithReference(rabbitMq);

builder.Build().Run();
```

This allows the payment team to work in isolation without starting all other services.

#### Step 4: Share Contracts

Create a `Contracts` project with shared DTOs. For example, a `PaymentCompletedEvent` that both the payment service and order service use. Reference this project from both services.

#### Step 5: Update CI Pipeline

Modify the CI pipeline to build both AppHosts and run integration tests for each. You may also want to build and push images only for the services that changed.

---

### 16.6 Summary

In this final chapter, we covered the organizational and operational best practices that turn a simple Aspire demo into a production‑ready, enterprise‑scale system. You learned:

- How to structure a large solution with multiple AppHosts, shared contracts, and reusable components.
- Versioning strategies for APIs, messages, and packages.
- How to set up CI/CD pipelines with GitHub Actions, including building, testing, and deploying to Azure Container Apps or Kubernetes.
- Common pitfalls and how to avoid them.

With these practices, you’re ready to apply .NET Aspire to your own real‑world projects. The journey from a distributed application novice to an expert is complete—but the learning never stops. Keep exploring, keep building, and may your services always be healthy.

---

**Exercises**

1. Refactor your e‑commerce solution to use a separate `Contracts` project. Move all shared DTOs and event classes into it. Update service references.
2. Set up a GitHub Actions workflow that builds the solution, runs integration tests (using the test AppHost), and deploys to a staging environment using `azd`.
3. Create a versioned API endpoint in the API service (e.g., `/api/v2/products`) that returns a different response. Use API versioning to route requests.
4. Implement a dead‑letter queue strategy for RabbitMQ messages that fail processing. Configure the worker to move messages to a DLQ after 3 retries.

---

## Appendix: Additional Resources

- [Official .NET Aspire Documentation](https://learn.microsoft.com/dotnet/aspire)
- [Aspire GitHub Repository](https://github.com/dotnet/aspire)
- [Aspirate (Community Tool)](https://github.com/prom3theu5/aspirate)
- [Azure Developer CLI (azd)](https://learn.microsoft.com/azure/developer/azure-developer-cli)

Thank you for reading *Building Cloud-Native Apps with .NET Aspire*. We hope this handbook has equipped you with the knowledge and confidence to build, deploy, and operate modern distributed applications with ease. Happy coding!

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='15. production_readiness_and_day_2_operations.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <span style='color:gray; font-size:1.05em;'>Next</span>
</div>
